<a href="https://colab.research.google.com/github/archiechang/study/blob/master/Signate/ch01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("ggplot")

In [ ]:
train=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Compete/Signate/ch01/data/train.csv")
test=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Compete/Signate/ch01/data/test.csv")

In [ ]:
train.describe()

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
count,8000.000000,8000.000000,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,4983.683000,0.513000,6.029750,5.92350,5.881875,0.538625,0.358125,17154.067750,17983.514000,0.491750
std,2893.028418,0.499862,2.623876,2.64892,3.189897,0.629728,0.479479,1820.796171,772.348402,0.499963
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,13475.000000,16650.000000,0.000000
25%,2473.750000,0.000000,4.000000,4.00000,3.000000,0.000000,0.000000,16544.000000,17256.000000,0.000000
50%,4985.500000,1.000000,6.000000,6.00000,6.000000,0.000000,0.000000,17409.000000,18021.000000,0.000000
75%,7493.250000,1.000000,8.000000,8.00000,8.000000,1.000000,1.000000,18274.000000,18472.000000,1.000000
max,9998.000000,1.000000,14.000000,14.00000,17.000000,2.000000,1.000000,20619.000000,20101.000000,1.000000


## 有益データ一覧

### blueFirstBlood
### blueKills:青チームによって殺された敵の数
### blueDeaths:青チームの死亡者数
### blueAssists
### blueEliteMonsters:青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）
### blueDragons
### blueTotalExperience:青チームの得た経験値合計


In [ ]:
#有益データのみ抽出する
#train_new=train[["blueFirstBlood","blueKills","blueDeaths","blueAssists","blueEliteMonsters","blueDragons","blueTotalExperience","blueWins"]]
#test_new=test[["blueFirstBlood","blueKills","blueDeaths","blueAssists","blueEliteMonsters","blueDragons","blueTotalExperience"]]
#train_new=train[["blueFirstBlood","blueKills","blueDeaths","blueAssists","blueEliteMonsters","blueDragons","blueTotalExperience"]]
#KD比率を追加する
#train_new["killAndDeaths"]=train_new["blueKills"]/train_new["blueDeaths"]
#test_new["killAndDeaths"]=test_new["blueKills"]/test_new["blueDeaths"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
train_x=train.drop(["blueWins"],axis=1)
train_y=train[["blueWins"]]

In [ ]:
train_x.corr()

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
gameId,1.000000,-0.001356,0.015867,0.011907,0.013440,0.001194,0.001223,0.012690,-0.000432
blueFirstBlood,-0.001356,1.000000,0.283559,-0.325833,0.241781,0.213862,0.155051,-0.111157,0.033651
blueKills,0.015867,0.283559,1.000000,-0.079929,0.710046,0.246184,0.196728,0.099387,0.309264
blueDeaths,0.011907,-0.325833,-0.079929,1.000000,-0.060368,-0.223138,-0.180798,0.060901,-0.051879
blueAssists,0.013440,0.241781,0.710046,-0.060368,1.000000,0.213466,0.176668,0.108863,0.301567
blueEliteMonsters,0.001194,0.213862,0.246184,-0.223138,0.213466,1.000000,0.787022,0.149436,0.295139
blueDragons,0.001223,0.155051,0.196728,-0.180798,0.176668,0.787022,1.000000,0.185465,0.295516
blueTotalGold,0.012690,-0.111157,0.099387,0.060901,0.108863,0.149436,0.185465,1.000000,0.162229
blueTotalExperience,-0.000432,0.033651,0.309264,-0.051879,0.301567,0.295139,0.295516,0.162229,1.000000


# モデルチューニング

In [ ]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold

In [ ]:
scores_accuracy=[]
scores_logloss=[]

kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(train_x):
    tr_x,va_x=train_x.iloc[tr_idx],train_x.iloc[va_idx]
    tr_y,va_y=train_y.iloc[tr_idx],train_y.iloc[va_idx]
    model=XGBClassifier(max_depth=12,min_child_weight=9.0,random_state=71)
    model.fit(tr_x,tr_y)

    va_pred=model.predict_proba(va_x)[:,1]
    logloss=log_loss(va_y,va_pred)
    accuracy=accuracy_score(va_y,va_pred>0.5)

    scores_accuracy.append(accuracy)
    scores_logloss.append(logloss)

logloss=np.mean(scores_logloss)
accuracy=np.mean(scores_accuracy)
print(f"logloss:{logloss:.4f},accuracy:{accuracy:.4f}")

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

logloss:0.4620,accuracy:0.7874


In [ ]:
import itertools
# チューニング候補とするパラメータを準備する
param_space={
    "max_depth":{7,8,9,10,11,12,13},
    "min_child_weight":{8.0,9.0,10.0,11.0}
}

#探索するハイパーパラメータの組み合わせ
param_combinations=itertools.product(param_space["max_depth"],param_space["min_child_weight"])

#各パラメータの組み合わせ、それに対するスコアを保存するリスト
params=[]
scores=[]

In [ ]:
#各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth,min_child_weight in param_combinations:

    score_folds=[]
    #クロスバリデーションを行う
    #学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf=KFold(n_splits=4,shuffle=True,random_state=123456)
    for tr_idx,va_idx in kf.split(train_x):
        #学習データを学習データとバリデーションに分ける
        tr_x,va_x=train_x.iloc[tr_idx],train_x.iloc[va_idx]
        tr_y,va_y=train_y.iloc[tr_idx],train_y.iloc[va_idx]

        #モデルの学習を行う
        model=XGBClassifier(n_estimators=20,random_state=71,max_depth=max_depth,min_child_weight=min_child_weight)
        model.fit(tr_x,tr_y)

        #バリデーションデータでのスコアを計算し、保存する
        va_pred=model.predict_proba(va_x)[:,1]
        logloss=log_loss(va_y,va_pred)
        score_folds.append(logloss)
    #各foldのスコアを平均する
    score_mean=np.mean(score_folds)

    #パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth,min_child_weight))
    scores.append(score_mean)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [ ]:
#最もスコアが良いものをベストなパラメータとする
best_idx=np.argsort(scores)[0]
best_param=params[best_idx]
print(f"max_depth: {best_param[0]}, min_child_weight:{best_param[1]}")
# max_depth=7,min_child_weight=4.0スコアが最も良かった

max_depth: 12, min_child_weight:9.0


# ロジスティクス特徴量作成

In [ ]:

# xgboostモデル
model_xgb=XGBClassifier(n_estimators=20,max_depth=9,min_child_weight=7.0,random_state=71)
model_xgb.fit(train_x,train_y)
pred_xgb=model_xgb.predict_proba(test_new)[:,1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

train_x2=train.drop(["blueWins"],axis=1)
train_y2=train[["blueWins"]]

sc=StandardScaler()
train_x_sc=sc.fit_transform(train_x2)

model_lr=LogisticRegression(solver="lbfgs",max_iter=300)
model_lr.fit(train_x_sc,train_y)
pred_lr=model_lr.predict_proba(test)[:,1]

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#予測値の加重平均をとる
pred=pred_xgb*1.0
#pred=pred_xgb*0.8+pred_lr*0.2
pred_label=np.where(pred>0.5,1,0)

In [ ]:
pred_label

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
print(model_lr.score(train_x_sc,train_y))

0.720375


In [ ]:
#提出用ファイルの作成
submission=pd.DataFrame({"gameId":test["gameId"],"Survived":pred_label})
submission.to_csv("submission.csv",index=False)